In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#카테고리를 원핫으로 해보고 모델좀 크게만들어보자 id빼버리고

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [60]:
traindata=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
testdata=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
data=pd.concat([traindata,testdata],axis=0,sort=False)

In [76]:
len(testdata)

1459

In [68]:
xtrain=data.drop('SalePrice',axis=1)
ytrain=data['SalePrice']
xtrain=xtrain.drop(['Id','Alley','PoolQC','Fence','MiscFeature','FireplaceQu'],axis=1)
trainmean=xtrain.mean()
trainmode=xtrain.mode()
xtrain=xtrain.fillna(trainmean)
xtrain=xtrain.fillna(trainmode.iloc[0])

xtrain=pd.get_dummies(xtrain)
for col in xtrain.select_dtypes('object') :
    xtrain[col]=pd.get_dummies(xtrain[col])
#xtrain['MSSubClass']=xtrain['MSSubClass'].astype('object')
#xtrain['MSSubClass']=pd.get_dummies(xtrain['MSSubClass'])
xtrain['OverallQual']=xtrain['OverallQual'].astype('category').cat.codes
xtrain['OverallCond']=xtrain['OverallCond'].astype('category').cat.codes
xtraincopy=xtrain.columns
scale=StandardScaler()
xtrain=scale.fit_transform(xtrain)

#xtrain=
# MSSubClass 16 OverallQual 10 OverallCond 10
xtrainuse=xtrain[:1460]
ytrainuse=ytrain[:1460]
xtest=xtrain[1460:]


In [69]:
xtraincopy

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=270)

In [43]:
xtrain

array([[ 0.07337496, -0.22937175, -0.20714171, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-0.87256276,  0.4519361 , -0.09188637, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.07337496, -0.09311018,  0.07347998, ..., -0.11785113,
         0.4676514 , -0.30599503],
       ...,
       [ 0.30985939, -0.18395123, -0.14781027, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-0.87256276, -0.09311018, -0.08016039, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-0.87256276,  0.22483348, -0.05811155, ..., -0.11785113,
         0.4676514 , -0.30599503]])

In [90]:
model=keras.models.Sequential()
model.add(keras.layers.Dense(512,input_shape=(xtrain.shape[1],),activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(256,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(150,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(32,activation='relu'))

model.add(keras.layers.Dense(1))
model.compile(loss='mse',optimizer=keras.optimizers.Adam(learning_rate=0.0001))
early=keras.callbacks.EarlyStopping(patience=10)


In [91]:
model.fit(xtrainuse,ytrainuse,epochs=1000,validation_split=0.2,callbacks=[early])

Epoch 1/1000
37/37 [==============================] - 1s 10ms/step - loss: 38372698543.1579 - val_loss: 39918120960.0000
Epoch 2/1000
37/37 [==============================] - 0s 6ms/step - loss: 40218167403.7895 - val_loss: 39915147264.0000
Epoch 3/1000
37/37 [==============================] - 0s 6ms/step - loss: 39783518746.9474 - val_loss: 39894601728.0000
Epoch 4/1000
37/37 [==============================] - 0s 6ms/step - loss: 38318177872.8421 - val_loss: 39769337856.0000
Epoch 5/1000
37/37 [==============================] - 0s 6ms/step - loss: 40604652058.9474 - val_loss: 39105478656.0000
Epoch 6/1000
37/37 [==============================] - 0s 6ms/step - loss: 40014610432.0000 - val_loss: 36351737856.0000
Epoch 7/1000
37/37 [==============================] - 0s 6ms/step - loss: 32747365376.0000 - val_loss: 27927934976.0000
Epoch 8/1000
37/37 [==============================] - 0s 6ms/step - loss: 22815234802.5263 - val_loss: 13491410944.0000
Epoch 9/1000
37/37 [===================

In [92]:
res=model.predict(xtest)

In [93]:
len(res)
outputdata=pd.DataFrame(res,columns=['SalePrice'])
outputdata['Id']=testdata['Id']
outputdata.to_csv('output.csv',index=False)